**Javier Rojas Herrera**

jrojash1995@gmail.com

**Hardware, Deployment y MLOps**



### Letra pequeña del Deep Learning

* **Entrenamiento e Inferencia requieren de hardware avanzado en computo numérico**
* Se requieren grandes volumenes de datos etiquetados



###  <center>[<img src="images/gpuvscpuvstpu.webp" width="80%"/> ](attachment:image.png)</center>

## GPU (graphics processing unit)
###  <center>[<img src="images/A100.jpg" width="60%"/> ](attachment:image.png)</center>

## GPU vs CPU
###  <center>[<img src="images/gpuvscpu.png" width="60%"/> ](attachment:image.png)</center>

## GPU vs CPU: Inferencia
###  <center>[<img src="images/gpuvscpu2.png" width="70%"/> ](attachment:image.png)</center>

## Tabla resumen GPUS

|GPU |  Cuda cores | Tensor cores | VRAM  | Power | Precio |
|----------|----------|----------| ----------|  ----------| ----------|
| T4    | 2500   | 320  | 15 GB | 70 W | 1100 usd |
| L4   | 7680   | 240  | 24 GB | 72 W | 2600 usd |
| L40   | 18176    | 568  | 48 GB | 300 W | 8400 usd |
| A100    | 6920   |  422   | 80 GB | 400 W | 12000 usd |
| H100    | 14592    |  456    | 80 GB | 350 W | 30000 usd |


In [1]:
!nvidia-smi

Thu Dec 21 15:42:07 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080        Off | 00000000:07:00.0 Off |                  N/A |
|  0%   44C    P8              23W / 320W |    224MiB / 10240MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from torchvision.models import vit_b_16 , ViT_B_16_Weights
from torchvision.transforms import v2
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader,Subset
import torch
import matplotlib.pyplot as plt
import time
from trans import UnNormalize

In [3]:
##load model VIT B 16
weights = ViT_B_16_Weights.DEFAULT
preprocess = weights.transforms()
model = vit_b_16(weights=weights)
model.heads.head = torch.nn.Linear(768,10)

##set optimizer and loss
optim = torch.optim.Adam(model.parameters())
cross_entropy = torch.nn.CrossEntropyLoss()

##load data into dataloader
data = CIFAR10("./", download=True, train=True, transform=weights.transforms())
data = CIFAR10("./", download=True, train=True, transform=weights.transforms())
subset_indices = torch.randperm(len(data))[:1000]
subset_cifar10 = Subset(data, subset_indices)
dataloader = DataLoader(subset_cifar10, batch_size=32, shuffle=False)

class_names = ["Airplane","Auto","Bird","Cat","Deer","Dog","Frog","Horse","Ship","Truck"]

Files already downloaded and verified
Files already downloaded and verified


In [4]:
std = weights.transforms().std
mean = weights.transforms().mean
invTrans=UnNormalize(mean,std)
@interact
def show_articles_more_than(x=1000):
    plt.figure(figsize=(5,3))
    print("Label: ",class_names[data[x][1]])
    plt.axis('off')
    plt.imshow(invTrans(data[x][0]).permute(1,2,0))

interactive(children=(IntSlider(value=1000, description='x', max=3000, min=-1000), Output()), _dom_classes=('w…

# ¿Qué elementos utilizan VRAM en un entrenamiento?

---

- Almacenamiento de tensores de entrada


- Almacenamiento de los parametros del modelo (weight and biases)

- Almacenamiento de gradientes (backpropagation)

- Almacenamiento de tensores de salida

---

In [5]:
## Tomando un batch del dataloader y transfiriendolo a VRAM
for image,label in dataloader:
    image_=image.cuda()
    break
!nvidia-smi

Thu Dec 21 15:46:42 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080        Off | 00000000:07:00.0 Off |                  N/A |
|  0%   45C    P2              24W / 320W |    463MiB / 10240MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
## transfiriendo el modelo a VRAM
model = model.cuda()
!nvidia-smi

Thu Dec 21 15:47:20 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080        Off | 00000000:07:00.0 Off |                  N/A |
|  0%   47C    P2              23W / 320W |    845MiB / 10240MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Locura de los parámetros


# <center>[ <img src="images/madness.png" width="60%"/>](attachment:image.png)</center>

## ¿Cómo aprovechar al maximo el hardware disponible?

# FP32 vs FP16

 # <center>[ <img src="images/fp16.ppm" width="80%"/>](attachment:image.png)</center>


## ¿Es posible usar representacion de FP16 para entrenar?

* Algunas operaciones como las convoluciones o lineales, pueden realizarse completamente en FP16
* Sin embargo, otras operaciones como la reducción, a menudo pueden necesitar la representacion en FP32

## Precision mixta (AMP)

 # <center>[ <img src="images/amp.png"/>](attachment:image.png)</center>

 # Tensor cores
 
 # <center>[ <img src="images/tensor_cores.gif"/>](attachment:image.png)</center>

 # <center>[ <img src="images/tensorop.png" width=100%/>](attachment:image.png)</center>

### Entrenamiento tradicional

In [7]:
for epoch in range(1):
    time_i=time.time()
    epoch_loss = 0.0
    for image,label in dataloader:
        optim.zero_grad() 
        image=image.cuda()  
        label=label.cuda()
        output = model(image)   
        loss= cross_entropy(output,label)        
        loss.backward()     
        epoch_loss+=loss.item()
        optim.step()
    print(f'Tiempo por epoca: {time.time()-time_i} segs | Epoch loss: {epoch_loss}')        

Tiempo por epoca: 10.426695585250854 segs | Epoch loss: 79.35196542739868


## Entrenamiento utilizando precision mixta + tensor cores

In [8]:
for epoch in range(1):
    time_i=time.time()
    epoch_loss = 0.0
    for image,label in dataloader:
        optim.zero_grad()
        image=image.cuda()
        label=label.cuda()
        with torch.autocast(device_type="cuda"):
            output = model(image)
            loss= cross_entropy(output,label)        
            loss.backward()
            optim.step()
        epoch_loss+=loss.item()
    print(f'Tiempo por epoca: {time.time()-time_i} segs | Epoch loss: {epoch_loss}')

Tiempo por epoca: 4.730200529098511 segs | Epoch loss: 72.21149444580078


## ¿Qué problemas pueden ocurrir al trabajar con una precisión de 16 bits?

* Cálculo de gradientes acumulativos podrian no poder representarse en FP16 (Desvanecimiento de gradiente)

### Entrenamiento con cálculo de gradiente escalado

In [9]:
scaler = torch.cuda.amp.GradScaler()  
for epoch in range(3):
    time_i=time.time()
    epoch_loss = 0.0
    for image,label in dataloader:
        optim.zero_grad()
        image=image.cuda()
        label=label.cuda()
        with torch.autocast(device_type="cuda"):
            output = model(image)
            loss= cross_entropy(output,label)        
        scaler.scale(loss).backward()
        scaler.step(optim)
        scaler.update()
        epoch_loss+=loss.item()
    print(f'Tiempo por epoca: {time.time()-time_i} segs | Epoch loss: {epoch_loss}')

Tiempo por epoca: 4.647127866744995 segs | Epoch loss: 67.93331909179688
Tiempo por epoca: 4.6152918338775635 segs | Epoch loss: 66.21737670898438
Tiempo por epoca: 4.61234712600708 segs | Epoch loss: 65.7242431640625


# ¿Qué sucede si no dispongo de hardware o si requiero de pocas horas de computo?

### Principales servicios cloud para creación de máquinas virtuales

# <center>[<img src="images/azurevs.jpg" width="80%"/>](attachment:image.png)</center>

### Pros de utilizar máquinas virtuales

* Fácil de crear y configurar según las necesidades

* Costo bajo al corto plazo

* Integración directa con otros servicios cloud del mismo prestador

### Contras de utilizar máquinas virtuales

* Los recursos solicitados pueden no estar disponibles

* Alto costo a largo plazo

## ¿Usar MV es lo más eficiente para realizar tareas de machine learning en la nube?

* Los modelos en MV no escalan (Inferencia)

* Entrenamiento y despliegue de modelos complejo de automatizar

# <center>[<img src="images/mlstudiovsvertex.png" width="60%"/>](attachment:image.png)</center>

### Ventajas al utilizar servicios especializados para ML en la nube

* Deployment escalable y automatizado de modelos

* Entrenamiento automatizado (pipelines)

* Disponibilidad de una familia de modelos pre entrenados a través de API

* Creación de notebooks jupyter

# <center>[<img src="images/mlsteps.jpg" width="80%"/>](attachment:image.png)</center>

## Deployment de modelos de ML

* Disponibilizar modelos para el uso real de usuarios

# <center>[<img src="images/depl.png" width="50%"/>](attachment:image.png)</center>

## Modelo como API
# <center>[<img src="images/apimodel.png" width="70%"/>](attachment:image.png)</center>

## Frameworks para deployment de modelos

# <center>[<img src="images/deploy.png" width="70%"/>](attachment:image.png)</center>

### Ejemplo:  Bento ML

In [10]:
import bentoml
bentoml.pytorch.save_model(
    "vit16",   # Model name
    model,  # objeto que contiene al modelo
)

Model(tag="vit16:owiia2vahcjejnbo", path="/home/javier/bentoml/models/vit16/owiia2vahcjejnbo/")

## Modelo como API
# <center>[<img src="images/depl2.png" width="70%"/>](attachment:image.png)</center>

## Problemas de levantar modelos API en MV

# <center>[<img src="images/apin2.png" width="70%"/>](attachment:image.png)</center>

## Solución: Escalar modelos API en cloud

# <center>[<img src="images/depl4.png" width="50%"/>](attachment:image.png)</center>

### Mostrar ejemplo de escalamiento en VERTEX AI

## ¿Qué es MLOps?

* Paradigma repetible que tiene como objetivo implementar y mantener modelos de aprendizaje automático en producción de manera confiable y eficiente.


# <center>[<img src="images/mlops.png" width="80%"/>](attachment:image.png)</center>

## Pipelines en MLOps

* Una Pipeline es un flujo de trabajo conformado por uno o varios componentes y sus interacciones a través de entradas y salidas.

# <center>[<img src="images/compo.png" width="60%"/>](attachment:image.png)</center>

# <center>[<img src="images/pipeline.png" width="60%"/>](attachment:image.png)</center>

### Frameworks para MLOps

# <center>[<img src="images/mlops_frame2.png" width="70%"/>](attachment:image.png)</center>

### Servicios cloud para MLOps
# <center>[<img src="images/mlstudiovsvertex.png" width="60%"/>](attachment:image.png)</center>

### Ejemplo de pipeline de juguete definida en Kubeflow

In [11]:
import kfp.dsl as dsl
from kfp.v2 import compiler

@dsl.component
def add(a: float, b: float) -> float:
    return a + b

@dsl.component
def mul(a: float, b: float) -> float:
    return a * b

@dsl.pipeline
def add_pipeline(a: float, b: float):
    add_task = add(a=a, b=b)
    mul_task = mul(a=a, b=add_task.output)
    
compiler.Compiler().compile(pipeline_func=add_pipeline, package_path='add_pipeline.json')


/tmp/ipykernel_10005/299293933.py:2: DeprecationWarning: The module `kfp.v2` is deprecated and will be removed in a futureversion. Please import directly from the `kfp` namespace, instead of `kfp.v2`.
  from kfp.v2 import compiler
/run/media/javier/miniconda3/envs/UC/lib/python3.10/site-packages/kfp/dsl/component_decorator.py:119: FutureWarning: Python 3.7 has reached end-of-life. The default base_image used by the @dsl.component decorator will switch from 'python:3.7' to 'python:3.8' on April 23, 2024. To ensure your existing components work with versions of the KFP SDK released after that date, you should provide an explicit base_image argument and ensure your component works as intended on Python 3.8.
  return component_factory.create_component_from_func(


### Ejemplo de Pipeline real en vertex AI

# <center>[<img src="images/vertex.png" width="56%"/>](attachment:image.png)</center>